NLP: Analyzing Review Text


In [1]:
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [4]:
import gzip
import ujson as json

with gzip.open('yelp_train_academic_dataset_review_reduced.json.gz') as f:
    data = [json.loads(line) for line in f]

ModuleNotFoundError: No module named 'ujson'

In [3]:
import gzip
import ujson as json

with gzip.open('yelp_dataset.tar') as f:
    data = [json.loads(line) for line in f]

ValueError: Expected object or value

In [5]:
stars = [row['stars'] for row in data]

Column Transformer class: 

In [10]:
import numpy as np
from sklearn import base
class ColumnSelectTransformer2(base.BaseEstimator, base.TransformerMixin):
    
    def __init__(self, col_names):
        self.col_names = col_names  # We will need these in transform()
    
    def fit(self, X, y=None):
        # This transformer doesn't need to learn anything about the data,
        # so it can just return self without any further processing
        return self
    
    def transform(self, X):
        # Return an array with the same number of rows as X and one
        # column for each in self.col_names
        return_list=[]
        for i in X:
            line_list=[]
            for j in self.col_names:
                line_list.append(i[j])
            if len(line_list)==1:
                line_list=line_list[0]
            return_list.append(line_list)      
        return  return_list

In [20]:
text = [row['text'] for row in data]
text

["I don't know what Dr. Goldberg was like before  moving to Arizona, but let me tell you, STAY AWAY from this doctor and this office. I was going to Dr. Johnson before he left and Goldberg took over when Johnson left. He is not a caring doctor. He is only interested in the co-pay and having you come in for medication refills every month. He will not give refills and could less about patients's financial situations. Trying to get your 90 days mail away pharmacy prescriptions through this guy is a joke. And to make matters even worse, his office staff is incompetent. 90% of the time when you call the office, they'll put you through to a voice mail, that NO ONE ever answers or returns your call. Both my adult children and husband have decided to leave this practice after experiencing such frustration. The entire office has an attitude like they are doing you a favor. Give me a break! Stay away from this doc and the practice. You deserve better and they will not be there when you really ne

In [11]:
#worked
import sklearn
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import GradientBoostingRegressor

bag_of_words_est = Pipeline([
    # Column selector (remember the ML project?)
    ('cst',ColumnSelectTransformer(['text'])),
    # Vectorizer
    ('cv',CountVectorizer()),   
    # Frequency filter (if necessary)
    
    # Regressor
    ('gbr', GradientBoostingRegressor(loss ='ls', max_depth=6)),
])
bag_of_words_est.fit(data, stars)


NameError: name 'ColumnSelectTransformer' is not defined

In [12]:
from sklearn.neural_network import MLPClassifier
bag_of_words_est = Pipeline([
    # Column selector (remember the ML project?)
    ('cst',ColumnSelectTransformer(['text'])),
    # Vectorizer
    ('cv',CountVectorizer()),   
    # Frequency filter (if necessary)
    
    # Regressor
    ('mlp', MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)),
])
bag_of_words_est.fit(data, stars)

NameError: name 'ColumnSelectTransformer' is not defined

In [33]:
cst = ColumnSelectTransformer2(['text'])
cat_column=cst.fit_transform(data)
count_vec=CountVectorizer().fit_transform(cat_column)
#CountVectorizer().fit_transform(cat_column)

["I don't know what Dr. Goldberg was like before  moving to Arizona, but let me tell you, STAY AWAY from this doctor and this office. I was going to Dr. Johnson before he left and Goldberg took over when Johnson left. He is not a caring doctor. He is only interested in the co-pay and having you come in for medication refills every month. He will not give refills and could less about patients's financial situations. Trying to get your 90 days mail away pharmacy prescriptions through this guy is a joke. And to make matters even worse, his office staff is incompetent. 90% of the time when you call the office, they'll put you through to a voice mail, that NO ONE ever answers or returns your call. Both my adult children and husband have decided to leave this practice after experiencing such frustration. The entire office has an attitude like they are doing you a favor. Give me a break! Stay away from this doc and the practice. You deserve better and they will not be there when you really ne

In [13]:
import sklearn
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import GradientBoostingRegressor

bag_of_words_est = Pipeline([  
    # Frequency filter (if necessary)
    
    # Regressor
    ('gbr', GradientBoostingRegressor(loss ='ls', max_depth=6)),
])
bag_of_words_est.fit(count_vec, stars)

NameError: name 'count_vec' is not defined

Finding food bigrams:

In [7]:
with gzip.open('yelp_train_academic_dataset_business.json.gz') as f:
    business_data = [json.loads(line) for line in f]
    

In [8]:
restaurants=[]
for business in business_data:
    if 'Restaurants' in business['categories']:
        restaurants.append(business)

In [9]:
restaurant_ids=[]
for i in restaurants:
    restaurant_ids.append(i['business_id'])

In [24]:
assert len(restaurant_ids) == 12876

In [25]:
data[0]

{'votes': {'funny': 0, 'useful': 0, 'cool': 0},
 'user_id': 'Qrs3EICADUKNFoUq2iHStA',
 'review_id': '_ePLBPrkrf4bhyiKWEn4Qg',
 'stars': 1,
 'date': '2013-04-19',
 'text': "I don't know what Dr. Goldberg was like before  moving to Arizona, but let me tell you, STAY AWAY from this doctor and this office. I was going to Dr. Johnson before he left and Goldberg took over when Johnson left. He is not a caring doctor. He is only interested in the co-pay and having you come in for medication refills every month. He will not give refills and could less about patients's financial situations. Trying to get your 90 days mail away pharmacy prescriptions through this guy is a joke. And to make matters even worse, his office staff is incompetent. 90% of the time when you call the office, they'll put you through to a voice mail, that NO ONE ever answers or returns your call. Both my adult children and husband have decided to leave this practice after experiencing such frustration. The entire office ha

The "business_id" here is the same as in the review data.  Use this to extract the review text for all reviews of restaurants.

In [10]:
restaurant_reviews=[]
restaurant_stars=[]
for i in data:
    if i['business_id'] in restaurant_ids:
        restaurant_reviews.append(i['text'])
        restaurant_stars.append(i['stars'])

In [14]:
CountVectorizer(max_features=300, 
                           ngram_range=(2,2), 
                           stop_words=STOP_WORDS.union({'apple', 'Apple'}))
import sklearn
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import GradientBoostingRegressor

bag_of_words_est = Pipeline([
    # Column selector (remember the ML project?)
    ('cst',ColumnSelectTransformer(['text'])),
    # Vectorizer
    ('cv',CountVectorizer(max_features=300, ngram_range=(2,2))),   
    # Frequency filter (if necessary)
    
    # Regressor
    ('gbr', GradientBoostingRegressor(loss ='ls', max_depth=6)),
])
#bag_of_words_est.fit(data, stars)

NameError: name 'STOP_WORDS' is not defined

In [12]:
word_counter=CountVectorizer(ngram_range=(1,1),min_df=10)
mywords=word_counter.fit(restaurant_reviews)
words=mywords.get_feature_names()
#counts=word_counter.fit_transform(restaurant_reviews).toarray().sum(axis=0)
word_counts=word_counter.fit_transform(restaurant_reviews).sum(axis=0)

In [13]:
dic_words={}
for i in range(len(words)):
    dic_words[words[i]]=word_counts.tolist()[0][i]+30

In [14]:
bigram_counter=CountVectorizer(ngram_range=(2,2), min_df=10,stop_words=STOP_WORDS,max_df=50)
mybigrams=bigram_counter.fit(restaurant_reviews)
bigrams=mybigrams.get_feature_names()
#counts=word_counter.fit_transform(restaurant_reviews).toarray().sum(axis=0)
bigram_counts=bigram_counter.fit_transform(restaurant_reviews).sum(axis=0)


In [32]:
dic_bigrams={}
for i in range(len(bigrams)):
    bigram_words=bigrams[i].split(' ')
    dic_bigrams[bigrams[i]]=bigram_counts.tolist()[0][i]/(dic_words[bigram_words[0]]*dic_words[bigram_words[1]])

In [37]:
top100=sorted(dic_bigrams.items(), key=lambda kv: kv[1], reverse=True)[:100]
top100=[x[0] for x in top100]
top100

['knick knacks',
 'rula bula',
 'himal chuli',
 'feng shui',
 'cien agaves',
 'ropa vieja',
 'tammie coe',
 'riff raff',
 'roka akor',
 'itty bitty',
 'khai hoan',
 'hoity toity',
 'baskin robbins',
 'reina pepiada',
 'chicha morada',
 'gulab jamun',
 'nooks crannies',
 'nanay gloria',
 'hodge podge',
 'luc lac',
 'dueling pianos',
 'haricot vert',
 'tutti santi',
 'patatas bravas',
 'nuoc mam',
 'hu tieu',
 'barnes noble',
 'puerto rican',
 'porta alba',
 'alain ducasse',
 'ore ida',
 'wal mart',
 'celine dion',
 'bradley ogden',
 'lomo saltado',
 'innis gunn',
 'krispy kreme',
 'vice versa',
 'holyrood 9a',
 'pura vida',
 'kao tod',
 'mccormick schmick',
 'valle luna',
 'deja vu',
 'chino bandido',
 'leaps bounds',
 'sous vide',
 'lloyd wright',
 'artery clogging',
 'har gow',
 'hors oeuvres',
 'bells whistles',
 'molecular gastronomy',
 'harry potter',
 'highs lows',
 'malai kofta',
 'ping pang',
 'aguas frescas',
 'ama ebi',
 'yada yada',
 'duct tape',
 'yadda yadda',
 'casey moore

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from spacy.lang.en.stop_words import STOP_WORDS
import sklearn
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import GradientBoostingRegressor
#mnb=MultinomialNB().fit(ng_counter)

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
from spacy.lang.en.stop_words import STOP_WORDS

ng_tfidf=TfidfVectorizer(max_features=100, 
                         ngram_range=(2,2), 
                         stop_words=STOP_WORDS,lowercase=True,min_df=1)
ng_tfidf=ng_tfidf.fit(restaurant_reviews[0:10])
#top100=ng_tfidf.get_feature_names()
#print(ng_tfidf.get_feature_names())
#trans_tfidf=ng_tfidf.transform(restaurant_reviews)

In [28]:
ng_tfidf.transform(restaurant_reviews[0:10])

<10x100 sparse matrix of type '<class 'numpy.float64'>'
	with 103 stored elements in Compressed Sparse Row format>

In [50]:
est = Pipeline([('vectorizer', TfidfVectorizer(max_features=100, 
                         ngram_range=(2,2), 
                         stop_words=STOP_WORDS,lowercase=True,min_df=10)), # min_df=50 because we're not interested in uncommon words
                ('classifier', MultinomialNB(alpha=.95))])#,class_prior=30))])
est.fit(restaurant_reviews,restaurant_stars)

#vocab = est.get_params()['vectorizer'].vocabulary_ # A dictionary of (word, value) pairs
                                                   # vocab['word'] is the index in the coefficient array corresponding to 'word'
    
#coeff_pos = est.get_params()['classifier'].feature_log_prob_[1] 
#coeff_neg = est.get_params()['classifier'].feature_log_prob_[0]

In [86]:
#score:0.02 max_df=100, min=10
#score:0.04 max_df=50, min=10
#score:0.05 max_df=50, min=10 + stop words
est = Pipeline([('vectorizer', TfidfVectorizer(max_features=100, 
                         ngram_range=(2,2), min_df=10,stop_words=STOP_WORDS,max_df=50)), # min_df=50 because we're not interested in uncommon words
                ('classifier', MultinomialNB(alpha=.95))])#,class_prior=30))])
est.fit(restaurant_reviews,restaurant_stars)

vocab = est.get_params()['vectorizer'].vocabulary_ # A dictionary of (word, value) pairs
                                                   # vocab['word'] is the index in the coefficient array corresponding to 'word'
    
#coeff_pos = est.get_params()['classifier'].feature_log_prob_[1] 
#coeff_neg = est.get_params()['classifier'].feature_log_prob_[0]

KeyboardInterrupt: 

In [91]:
a=CountVectorizer(max_features=100, 
                         ngram_range=(2,2), 
                         stop_words=STOP_WORDS,lowercase=True,min_df=1).fit(restaurant_reviews[0:2])
b=a.transform(restaurant_reviews[0:2])
a.get_feature_names()
#print(b)

['ago great',
 'allowed smoking',
 'atmosphere improved',
 'change law',
 'cigarette change',
 'great pancakes',
 'half ago',
 'law sure',
 'left smelling',
 'like cigarette',
 'like lot',
 'lizards ll',
 'll love',
 'lot lizards',
 'love pine',
 'pancakes problem',
 'pine cone',
 'problem time',
 'smelling like',
 'smoking left',
 'sure atmosphere',
 'time allowed',
 'went year',
 'year half']

In [89]:
restaurant_reviews[0:2]

["If you like lot lizards, you'll love the Pine Cone!",
 "Only went here once about a year and a half ago, but they had great pancakes! My only problem with it at the time was that they allowed smoking, so I left smelling like a cigarette. With the change in law, I'm sure the atmosphere has improved!"]

In [76]:
#worked
top100 = est.named_steps['vectorizer'].get_feature_names()
top100

['apple juice',
 'asian buffet',
 'awesome awesome',
 'banana split',
 'beer cheese',
 'bison burger',
 'bit ly',
 'bland bland',
 'busy busy',
 'butter jelly',
 'carnival world',
 'casa grande',
 'char siu',
 'cheesy bread',
 'chelsea kitchen',
 'china poblano',
 'chocolate croissant',
 'com user_local_photos',
 'corn dog',
 'country potatoes',
 'cous cous',
 'cracked egg',
 'cracker barrel',
 'cream soda',
 'da vang',
 'de lys',
 'el pollo',
 'fish cakes',
 'fix menu',
 'gift cards',
 'ginger ale',
 'grande orange',
 'halo halo',
 'har gow',
 'herb box',
 'high chair',
 'hot links',
 'http bit',
 'island style',
 'italian sub',
 'japanese curry',
 'jean philippe',
 'jersey mike',
 'joe farm',
 'kalua pig',
 'kalua pork',
 'kim chi',
 'la grande',
 'lake mead',
 'le cirque',
 'le thai',
 'legs prime',
 'lobster tacos',
 'los dos',
 'love bacon',
 'meat jun',
 'mellow mushroom',
 'mmm mmm',
 'mothers day',
 'noodle bowl',
 'ocean club',
 'oh oh',
 'pad kee',
 'pane bianco',
 'papa john

In [74]:
import spacy
nlp = spacy.load("en")
def tokenize_lemma(text):
    return [w.lemma_ for w in nlp(text)] 

stop_words_lemma = set(w.lemma_ for w in nlp(' '.join(STOP_WORDS)))
est = Pipeline([('vectorizer', TfidfVectorizer(max_features=100,
                         ngram_range=(2,2), min_df=10,stop_words=STOP_WORDS,max_df=50)), # min_df=50 because we're not interested in uncommon words
                ('classifier', MultinomialNB(alpha=10))])#,class_prior=30))])
est.fit(restaurant_reviews,restaurant_stars)

vocab = est.get_params()['vectorizer'].vocabulary_


In [75]:

top100 = est.named_steps['vectorizer'].get_feature_names()[::-1]

In [17]:
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin

class AdHocFeatures(BaseEstimator, TransformerMixin):
    """
    Given a keyword (e.g., "apple"), will transform documents into an
    encoding of several ad hoc features of each occurrences of the keyword:
        - If the keyword is capitalized
        - If it is plural
        - If it is possessive (in the stupid sense of being followed by 's)
        - If the keyword is a verb (e.g., for Ford vs ford)
    """

    def __init__(self, keyword):
        self.keyword = nlp(keyword)[0].lemma_
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return np.asarray([self.transform_doc(x) for x in X])
    
    def feature_posessive(self, doc):
        ## N.B. spaCy will tokenize "Apple's" as ["Apple", "'s"]
        hits = [i for i, word in enumerate(doc) if word.lemma_ == self.keyword]
        return sum((i + 1) < len(doc) and doc[i+1].text == "'s" for i in hits)
    
    def transform_doc(self, row):
        doc = nlp(row)
        words = [word for word in doc if word.lemma_ == self.keyword]
        return [sum(word.is_title for word in words),
                sum(word.tag_ in ('NNS', 'NNPS') for word in words),
                self.feature_posessive(doc),
                sum(word.pos_ == 'VERB' for word in words)]

In [40]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline, FeatureUnion
import spacy
nlp = spacy.load("en")

def tokenize_lemma(text):
    return [w.lemma_ for w in nlp(text)]
stop_words_lemma = set(w.lemma_ for w in nlp(' '.join(STOP_WORDS)))

features = FeatureUnion([('stem_vectorizer',
                          TfidfVectorizer(max_features=100, ngram_range=(2,2),
                                          stop_words=stop_words_lemma,
                                          tokenizer=tokenize_lemma,min_df=2,max_df=200))
                        ])
pipe = Pipeline([('features', features),
                 ('classifier', MultinomialNB(alpha=0))])


In [41]:

pipe.fit(restaurant_reviews[:10],restaurant_stars[:10])

vocab = pipe.named_steps['features'].get_params()['stem_vectorizer'].vocabulary_ 

/opt/conda/envs/data3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['-pron-'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
/opt/conda/envs/data3/lib/python3.6/site-packages/sklearn/naive_bayes.py:480: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


In [42]:
vocab

{'pine cone': 7,
 '.  ': 2,
 '. \n\n': 1,
 '.  \n\n': 3,
 'truck stop': 9,
 'delicious .': 4,
 'good !': 6,
 '!  ': 0,
 'food .': 5,
 'place breakfast': 8}

In [47]:
print(pipe.fit(restaurant_reviews[:10],restaurant_stars[:10]))

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=None,
       transformer_list=[('stem_vectorizer', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=200, max_features=100, min_df=2,
 ...ransformer_weights=None)), ('classifier', MultinomialNB(alpha=0, class_prior=None, fit_prior=True))])


/opt/conda/envs/data3/lib/python3.6/site-packages/sklearn/naive_bayes.py:480: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


In [39]:
restaurant_reviews[:10]

["If you like lot lizards, you'll love the Pine Cone!",
 "Only went here once about a year and a half ago, but they had great pancakes! My only problem with it at the time was that they allowed smoking, so I left smelling like a cigarette. With the change in law, I'm sure the atmosphere has improved!",
 'Ate a Saturday morning breakfast at the Pine Cone.  Friendly,quick service, normal prices.  But for the new year - the special was all-you-can-eat pancakes for $2.12, I added a breakfast sausage patty ($3.00).  Father had the typical, two eggs, toast, hash-bowns (seem fresh cut), sausage links.  Food was tasty, no major qualms other than - the hash-browns need a bit more cooking and the coffee needs a bit more bite.\n\nvery nice assortment of bakes goods - massive - as large as your head cream-puffs, eclairs - I took a blueberry muffin to go - which I enjoyed.  \n\nQuick exit from I-94...just northeast of  Madison',
 'This is definitely not your usual truck stop. Pastries look deliciou

In [115]:
est = TfidfVectorizer(max_features=100, 
                         ngram_range=(2,2), min_df=10,stop_words=STOP_WORDS,max_df=50)
est.fit(restaurant_reviews)

vocab = est.get_params().vocabulary_ #

AttributeError: 'dict' object has no attribute 'vocabulary_'

In [116]:
est.get_params()

{'analyzer': 'word',
 'binary': False,
 'decode_error': 'strict',
 'dtype': numpy.float64,
 'encoding': 'utf-8',
 'input': 'content',
 'lowercase': True,
 'max_df': 50,
 'max_features': 100,
 'min_df': 10,
 'ngram_range': (2, 2),
 'norm': 'l2',
 'preprocessor': None,
 'smooth_idf': True,
 'stop_words': {'a',
  'about',
  'above',
  'across',
  'after',
  'afterwards',
  'again',
  'against',
  'all',
  'almost',
  'alone',
  'along',
  'already',
  'also',
  'although',
  'always',
  'am',
  'among',
  'amongst',
  'amount',
  'an',
  'and',
  'another',
  'any',
  'anyhow',
  'anyone',
  'anything',
  'anyway',
  'anywhere',
  'are',
  'around',
  'as',
  'at',
  'back',
  'be',
  'became',
  'because',
  'become',
  'becomes',
  'becoming',
  'been',
  'before',
  'beforehand',
  'behind',
  'being',
  'below',
  'beside',
  'besides',
  'between',
  'beyond',
  'both',
  'bottom',
  'but',
  'by',
  'ca',
  'call',
  'can',
  'cannot',
  'could',
  'did',
  'do',
  'does',
  'doing'